In [47]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/abo-vqa/abo-images-small.tar.xz
/kaggle/input/fine-tuned/fine_tune_blip_lora/adapter_model.safetensors
/kaggle/input/fine-tuned/fine_tune_blip_lora/preprocessor_config.json
/kaggle/input/fine-tuned/fine_tune_blip_lora/adapter_config.json
/kaggle/input/fine-tuned/fine_tune_blip_lora/README.md
/kaggle/input/fine-tuned/fine_tune_blip_lora/tokenizer.json
/kaggle/input/fine-tuned/fine_tune_blip_lora/tokenizer_config.json
/kaggle/input/fine-tuned/fine_tune_blip_lora/special_tokens_map.json
/kaggle/input/fine-tuned/fine_tune_blip_lora/vocab.txt


In [3]:
pip install bitsandbytes && pip install git+https://github.com/salesforce/LAVIS.git && pip install git+https://github.com/OFA-Sys/OFA.git

  Cloning https://github.com/salesforce/LAVIS.git to /tmp/pip-req-build-l6i04k_7
  Running command git clone --filter=blob:none --quiet https://github.com/salesforce/LAVIS.git /tmp/pip-req-build-l6i04k_7
  Resolved https://github.com/salesforce/LAVIS.git to commit 506965b9c4a18c1e565bd32acaccabe0198433f7
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached contexttimer-0.3.3.tar.gz (4.9 kB)
  Preparing metadata (setup.py) ... done
  Using cached decord-0.6.0-py3-none-manylinux2010_x86_64.whl.metadata (422 bytes)
  Using cached diffusers-0.16.0-py3-none-any.whl.metadata (19 kB)
  Using cached fairscale-0.4.4.tar.gz (235 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached ftfy-6.3.1-py3-none-any.whl.metadata (7.3 kB)
  Using cached iopath-0.1.10.tar.

In [48]:
!pip install bert-score

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [49]:
import os
import pandas as pd
from PIL import Image
from torch.utils.data import Dataset, DataLoader
import torch
from transformers import Blip2Processor, BlipForConditionalGeneration
from peft import get_peft_model, LoraConfig, TaskType

In [50]:
! ls /kaggle/working/

extracted_abo_images  fine_tune_blip_lora  state.db


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [51]:
! ls /kaggle/working/fine_tune_blip_lora/

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


adapter_config.json	   README.md		    tokenizer_config.json
adapter_model.safetensors  special_tokens_map.json  tokenizer.json
preprocessor_config.json   THIS_IS_JUST_A_FLAG	    vocab.txt


In [52]:
! mkdir /kaggle/working/fine_tune_blip_lora/THIS_IS_JUST_A_FLAG

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


mkdir: cannot create directory ‘/kaggle/working/fine_tune_blip_lora/THIS_IS_JUST_A_FLAG’: File exists


In [53]:
!cp -r "/kaggle/input/fine-tuned/fine_tune_blip_lora"  "/kaggle/working/."

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [54]:
# ---- Configuration ----
CSV_FOLDER = "/kaggle/working/extracted_abo_images/abo-images-small/images/dataset_csv"
IMAGE_DIR = "/kaggle/working/extracted_abo_images/abo-images-small/images/small"
MODEL_NAME = "Salesforce/blip-vqa-base"
OUTPUT_DIR = "/kaggle/working/fine_tune_blip_lora"
BATCH_SIZE = 8
NUM_EPOCHS = 3
LEARNING_RATE = 5e-5
device = "cuda" if torch.cuda.is_available() else "cpu"

In [55]:
df = pd.concat([
    pd.read_csv(os.path.join(CSV_FOLDER, f))
    for f in os.listdir(CSV_FOLDER) if f.endswith(".csv")
])

print(f"Loaded {len(df)} rows from CSVs")
df = df[df['answer'].notnull()]
df = df[df['image_id'].notnull()]
df = df[df['path'].notnull()]
df = df[df['question'].notnull()]
print(f"New {len(df)} rows from CSVs")

Loaded 80336 rows from CSVs
New 79990 rows from CSVs


In [1]:
# === Initial Setup ===
import os
import pandas as pd
from PIL import Image
import torch
from tqdm import tqdm
from bert_score import score as bert_score
import warnings
warnings.filterwarnings('ignore')

# Configuration
CSV_FOLDER = "/kaggle/working/extracted_abo_images/abo-images-small/images/dataset_csv"
IMAGE_DIR = "/kaggle/working/extracted_abo_images/abo-images-small/images/small"
BATCH_SIZE = 8
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# Load dataset
print("Loading dataset...")
df = pd.concat([
    pd.read_csv(os.path.join(CSV_FOLDER, f))
    for f in os.listdir(CSV_FOLDER) if f.endswith(".csv")
])
print(f"Initial rows: {len(df)}")

# Clean data
df = df[df['answer'].notnull() & df['image_id'].notnull() & df['path'].notnull() & df['question'].notnull()]
print(f"Clean rows: {len(df)}")

# For testing, let's take a small sample (remove this for full run)
df = df.sample(100, random_state=42).reset_index(drop=True)
print(f"Working with sample of {len(df)} rows")

# === Model Loading Functions ===
def load_blip_model():
    print("\nLoading BLIP model...")
    from transformers import BlipProcessor, BlipForQuestionAnswering
    processor = BlipProcessor.from_pretrained("Salesforce/blip-vqa-base")
    model = BlipForQuestionAnswering.from_pretrained("Salesforce/blip-vqa-base").to(device)
    return processor, model

def load_blip2_model():
    print("\nLoading BLIP-2 model...")
    from transformers import Blip2Processor, Blip2ForConditionalGeneration
    processor = Blip2Processor.from_pretrained("Salesforce/blip2-opt-2.7b")
    model = Blip2ForConditionalGeneration.from_pretrained("Salesforce/blip2-opt-2.7b", torch_dtype=torch.float16).to(device)
    return processor, model

def load_ofa_model():
    print("\nLoading OFA model...")
    from transformers import OFATokenizer, OFAModel
    from transformers.models.ofa.generate import sequence_generator
    tokenizer = OFATokenizer.from_pretrained("OFA-Sys/OFA-medium")
    model = OFAModel.from_pretrained("OFA-Sys/OFA-medium", use_cache=True).to(device)
    return tokenizer, model

def load_vilbert_model():
    print("\nLoading ViLBERT model...")
    from transformers import ViltProcessor, ViltForQuestionAnswering
    processor = ViltProcessor.from_pretrained("dandelin/vilt-b32-finetuned-vqa")
    model = ViltForQuestionAnswering.from_pretrained("dandelin/vilt-b32-finetuned-vqa").to(device)
    return processor, model

def load_clip_model():
    print("\nLoading CLIP model...")
    from transformers import CLIPProcessor, CLIPModel
    processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
    model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to(device)
    return processor, model

# === Model Inference Functions ===
def run_blip_inference(processor, model, image, question):
    inputs = processor(image, question, return_tensors="pt").to(device)
    with torch.no_grad():
        outputs = model.generate(**inputs)
    return processor.decode(outputs[0], skip_special_tokens=True)

def run_blip2_inference(processor, model, image, question):
    inputs = processor(image, question, return_tensors="pt").to(device, torch.float16)
    with torch.no_grad():
        outputs = model.generate(**inputs)
    return processor.decode(outputs[0], skip_special_tokens=True)

def run_ofa_inference(tokenizer, model, image, question):
    inputs = tokenizer([question], return_tensors="pt").input_ids
    img = Image.open(image_path).convert("RGB")
    patch_img = tokenizer(img, return_tensors="pt").to(device)
    with torch.no_grad():
        outputs = model.generate(inputs.to(device), patch_images=patch_img["pixel_values"].to(device))
    return tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]

def run_vilbert_inference(processor, model, image, question):
    encoding = processor(image, question, return_tensors="pt").to(device)
    with torch.no_grad():
        outputs = model(**encoding)
    logits = outputs.logits
    idx = logits.argmax(-1).item()
    return model.config.id2label[idx]

def run_clip_inference(processor, model, image, question):
    # CLIP isn't a VQA model, but we can use it for zero-shot classification
    # This is a simplified approach - not ideal for VQA
    inputs = processor(text=[question], images=image, return_tensors="pt", padding=True).to(device)
    with torch.no_grad():
        outputs = model(**inputs)
    logits_per_image = outputs.logits_per_image
    probs = logits_per_image.softmax(dim=1)
    return "CLIP-not-proper-VQA"  # Simplified for demo

# === Main Evaluation Loop ===
def evaluate_models(df, models_to_run):
    results = df.copy()
    
    for model_name, (load_func, run_func) in models_to_run.items():
        print(f"\n=== Evaluating {model_name} ===")
        try:
            # Load model
            processor, model = load_func()
            
            # Run inference
            predictions = []
            for _, row in tqdm(df.iterrows(), total=len(df), desc=f"Running {model_name}"):
                image_path = os.path.join(IMAGE_DIR, row["path"])
                try:
                    image = Image.open(image_path).convert("RGB")
                    pred = run_func(processor, model, image, row["question"])
                    predictions.append(pred)
                except Exception as e:
                    print(f"Error processing {image_path}: {str(e)}")
                    predictions.append("ERROR")
            
            # Add predictions to results
            results[f"{model_name}_predicted"] = predictions
            
            # Compute BERTScore
            P, R, F1 = bert_score(
                cands=predictions,
                refs=results['answer'].tolist(),
                lang="en"
            )
            
            print(f"\n{model_name} BERTScore:")
            print(f" - Precision: {P.mean():.4f}")
            print(f" - Recall:    {R.mean():.4f}")
            print(f" - F1 Score:  {F1.mean():.4f}")
            
            # Free up memory
            del processor, model
            torch.cuda.empty_cache()
            
        except Exception as e:
            print(f"Failed to evaluate {model_name}: {str(e)}")
            results[f"{model_name}_predicted"] = ["ERROR"] * len(df)
    
    return results

# === Define Models to Evaluate ===
models_to_run = {
    "BLIP": (load_blip_model, run_blip_inference),
    "BLIP2": (load_blip2_model, run_blip2_inference),
    "OFA": (load_ofa_model, run_ofa_inference),
    "ViLBERT": (load_vilbert_model, run_vilbert_inference),
    "CLIP": (load_clip_model, run_clip_inference)
}

# === Run Evaluation ===
results_df = evaluate_models(df, models_to_run)

# === Save Results ===
output_path = "vqa_model_comparison_results.csv"
results_df.to_csv(output_path, index=False)
print(f"\nResults saved to {output_path}")

# === Summary Report ===
print("\n=== Final Summary ===")
for model_name in models_to_run.keys():
    col_name = f"{model_name}_predicted"
    if col_name in results_df.columns:
        correct = (results_df[col_name].str.lower() == results_df['answer'].str.lower()).sum()
        accuracy = correct / len(results_df)
        print(f"{model_name} Accuracy: {accuracy:.2%}")

Using device: cuda
Loading dataset...
Initial rows: 80336
Clean rows: 79990
Working with sample of 100 rows

=== Evaluating BLIP ===

Loading BLIP model...


2025-05-17 11:50:42.384675: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747482642.407432     243 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747482642.414680     243 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
Running BLIP: 100%|██████████| 100/100 [00:10<00:00,  9.79it/s]


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/4.19k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/605M [00:00<?, ?B/s]


Running CLIP: 100%|██████████| 100/100 [00:03<00:00, 31.20it/s][A
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



🔍 CLIP BERTScore:
 - Precision: 0.7797
 - Recall:    0.8433
 - F1 Score:  0.8098

Results saved to vqa_model_comparison_results.csv

